<a href="https://colab.research.google.com/github/BrunoRBorges/Challenge_Data_Science/blob/main/Challenge_data_science_s03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge Data Science Alura - Semana 03
Bem-vindo ao meu GitHub! Meu nome é Bruno, e gostaria de lhe apresentar um pouco do trabalho que desenvolvi durante este Challenge!

A partir de agora, irei utilizar os dados com colunas dummy gerados na semana 02 para criar um recomendador de imóveis, baseado numa ID passada pelo usuário.

In [1]:
!pip install pyspark
from google.colab import drive

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 65.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8fda746099b394a7385638bc061340423fb21a3a706cef0ac836dd0115fee131
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Mounted at /content/drive


In [2]:
# Importa a sessão Spark e os SparkTypes
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType

In [3]:
# Cria a sessão Spark (lembrar de fechar a sessão depois)
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Challenge Semana 03") \
    .getOrCreate()

In [4]:
#Lê o arquivo de dados da semana anterior
dataset_imoveis = spark.read.parquet('/content/drive/MyDrive/Challenge Data Science/Semana 2/output_parquet_dataframe')

In [5]:
# Iniciando a etapa de Machine Learning

from pyspark.ml.stat import Correlation
import pandas as pd
from pyspark.ml.feature import VectorAssembler

In [6]:
# Renomeando as colunas para a vetorização

dataset_imoveis = dataset_imoveis.withColumnRenamed('valor','label')

#Vetorização

In [7]:
# Criando a lista de variáveis de input X
X = dataset_imoveis.columns
X.remove('id')
X.remove('tipo_anuncio')
X.remove('tipo_unidade')
X.remove('tipo_uso')
X.remove('bairro')
X.remove('tipo')
X.remove('label')
X

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'Condomínio fechado',
 'Playground',
 'Portão eletrônico',
 'Piscina',
 'Animais permitidos',
 'Portaria 24h',
 'vazio',
 'Elevador',
 'Academia',
 'Salão de festas',
 'Churrasqueira',
 'Zona Norte',
 'Zona Oeste',
 'Zona Central',
 'Zona Sul']

In [8]:
# Criando o Assemble de vetorização

assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [9]:
# Vetorizando o Dataframe

dataset_imoveis_vetorizado = assembler.transform(dataset_imoveis)

#Redução de Dimensionalidade com PCA e StandardScaler

In [10]:
# Redução de dimensionalidade dos hiperparametros
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(dataset_imoveis_vetorizado)
dados_imoveis_scaled = scaler_model.transform(dataset_imoveis_vetorizado)

In [11]:
# Aplicando o método PCA
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(dados_imoveis_scaled)
dados_imoveis_pca = model_pca.transform(dados_imoveis_scaled)

#Utilizando o Kmeans para separar os clusters

In [12]:
# Utilizando o Kmeans para fazer o sistema de recomendação de imóveis

from pyspark.ml.clustering import KMeans

SEED = 1224
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca').setK(5).setSeed(SEED)
model_kmeans = kmeans.fit(dados_imoveis_pca)
dados_imoveis_kmeans = model_kmeans.transform(dados_imoveis_pca)

In [13]:
# Filtro de imóveis do mesmo cluster

dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 1).show(10)

+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------+----------+-----+-----+--------+------------------+----------+-----------------+-------+------------------+------------+-----+--------+--------+---------------+-------------+----------+----------+------------+--------+--------------------+--------------------+--------------------+-----------+
|                  id|andar|area_util|banheiros|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|        bairro|condominio| iptu| tipo|   label|Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|vazio|Elevador|Academia|Salão de festas|Churrasqueira|Zona Norte|Zona Oeste|Zona Central|Zona Sul|            features|     scaled_features|        pca_features|cluster_pca|
+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------+----------+-----+-----+--------+-----------------

In [14]:
dados_imoveis_kmeans = dados_imoveis_kmeans.drop('vazio')
dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 2).show(n=10, truncate = False)

+------------------------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+-----------------------+----------+------+-----+--------+------------------+----------+-----------------+-------+------------------+------------+--------+--------+---------------+-------------+----------+----------+------------+--------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|id  

In [15]:
# Calculando a distância euclidiana dos imóveis

from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType

cluster = 2

id_teste = '406fc037-8af7-4cd2-8231-f2ba75ede4ca'

imoveis_recomendados = dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == cluster)\
                                       .select('bairro', 'id', 'pca_features')
                                       
vetores_valor_imoveis = imoveis_recomendados.filter(imoveis_recomendados.id == id_teste)\
                                          .select('pca_features').collect()[0][0]

# Define função que calcula a distância entre imóveis no espaço criado pelos parâmetros.

In [16]:
# Definindo função de cálculo de distância

def calcula_distancia(value):
  return euclidean(vetores_valor_imoveis, value)

In [17]:
vetores_valor_imoveis

DenseVector([-0.9218, 0.4126, 2.3725, 0.2908, 0.0266, 1.0379, -0.4888, 0.4492, 0.0301, -0.3035])

# Define a função recomendador

In [18]:
from pyspark.sql.functions import udf

def recomendador_imoveis(imovel_id):
  cluster = dados_imoveis_kmeans.filter(dados_imoveis_kmeans.id == imovel_id).select('cluster_pca').collect()[0][0]
  imoveis_recomendados = dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == cluster)\
                                              .select('bairro', 'id', 'pca_features')                
  vetores_valor_imoveis = imoveis_recomendados.filter(imoveis_recomendados.id == imovel_id)\
                                              .select('pca_features').collect()[0][0]
  udf_distancia_calculada = udf(calcula_distancia, FloatType())

  imoveis_recomendados_dist = imoveis_recomendados.withColumn('Dist', udf_distancia_calculada('pca_features'))

  recomendadas = spark.createDataFrame(imoveis_recomendados_dist.sort('Dist').take(10)).select(['bairro', 'id', 'Dist'])

  recomendadas.show(truncate=False)

In [19]:
recomendador_imoveis('406fc037-8af7-4cd2-8231-f2ba75ede4ca')

+----------+------------------------------------+--------------------+
|bairro    |id                                  |Dist                |
+----------+------------------------------------+--------------------+
|Copacabana|406fc037-8af7-4cd2-8231-f2ba75ede4ca|0.0                 |
|Copacabana|995eaac4-80b2-4dc0-b75f-bf3eb0569c95|0.00117090146522969 |
|Copacabana|f9f22d5c-4f4f-4e80-b6a2-af3ccf4b6e0c|0.005570203065872192|
|Copacabana|e61edd0a-1ee4-4275-880e-ea339492df99|0.05017727240920067 |
|Copacabana|350bc21e-b0bb-4237-a214-a2d44dfbfba5|0.06803227961063385 |
|Copacabana|87076407-0bd6-4fc0-a71b-4162c07d4dd2|0.06804215162992477 |
|Copacabana|95dc02a0-50e7-412a-9d1a-d094c2e519c7|0.06869017332792282 |
|Copacabana|51e1d4b9-d2fd-43bd-b34b-7b2399b2ffb2|0.06908772140741348 |
|Flamengo  |e80eb0e9-f1bc-487d-9c78-998c42db7f96|0.06994132697582245 |
|Copacabana|8c025393-6e6b-4573-87e0-0f9d47672e21|0.0832333192229271  |
+----------+------------------------------------+--------------------+



In [20]:
dataset_imoveis.show(n=5, truncate=False)

+------------------------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+------------+----------+------+-----+--------+------------------+----------+-----------------+-------+------------------+------------+-----+--------+--------+---------------+-------------+----------+----------+------------+--------+
|id                                  |andar|area_util|banheiros|quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro      |condominio|iptu  |tipo |label   |Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|vazio|Elevador|Academia|Salão de festas|Churrasqueira|Zona Norte|Zona Oeste|Zona Central|Zona Sul|
+------------------------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+------------+----------+------+-----+--------+------------------+----------+-----------------+-------+------------------+------------+-----+--------+--------+----------

In [21]:
recomendador_imoveis('03a386b6-7ab8-4eff-891d-f8a16efc1989')

+------------+------------------------------------+------------------+
|bairro      |id                                  |Dist              |
+------------+------------------------------------+------------------+
|Copacabana  |0a085dd9-c26e-4292-aad3-bc2ec0af2d8c|3.6586050987243652|
|Leblon      |8c07ae9e-38db-4d00-8e95-7559d876cab2|3.672654867172241 |
|Leblon      |f4e9821d-e149-4c8d-9faf-6713d025f9b4|3.694117307662964 |
|Laranjeiras |70ee9a62-1e8d-433b-93a3-db28c0f692d0|3.7030081748962402|
|Laranjeiras |e5b09b28-8ad1-4e57-a23f-33ed856c38fd|3.8990461826324463|
|Copacabana  |05128952-0422-4e6e-8b41-aebf27aa602d|3.9733035564422607|
|Flamengo    |d0190116-9d94-45e5-b157-4a8f74f33918|3.973332166671753 |
|Campo Grande|9c0039db-a54f-470d-a52f-f9d89dc31fea|3.9744839668273926|
|Campo Grande|77cbb46a-438d-4b17-bc69-35b55f25243f|3.9750795364379883|
|Humaitá     |efb55e0c-9c40-4a53-bef3-da69e787d039|3.9999611377716064|
+------------+------------------------------------+------------------+



#Desafio Extra! - Criando um pipeline
Foi utilizado para o Pipeline as informações geradas pelo VectorAssembler,StandardScaler e PCA.

In [23]:
# Desafio Extra! Criando um Pipeline

from pyspark.ml import Pipeline

pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='scaled_features'),
                                PCA(k=10, inputCol='scaled_features', outputCol='pca_features')])

In [25]:
pca_pipeline_imoveis = pca_pipeline.fit(dataset_imoveis)
dataset_imoveis_pipeline = pca_pipeline_imoveis.transform(dataset_imoveis)
dataset_imoveis_pipeline.show(5)

+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+------------+----------+------+-----+--------+------------------+----------+-----------------+-------+------------------+------------+-----+--------+--------+---------------+-------------+----------+----------+------------+--------+--------------------+--------------------+--------------------+
|                  id|andar|area_util|banheiros|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|      bairro|condominio|  iptu| tipo|   label|Condomínio fechado|Playground|Portão eletrônico|Piscina|Animais permitidos|Portaria 24h|vazio|Elevador|Academia|Salão de festas|Churrasqueira|Zona Norte|Zona Oeste|Zona Central|Zona Sul|            features|     scaled_features|        pca_features|
+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+------------+----------+------+-----+--------+------------------+----------+--------------

#Desafio Extra! - Avaliando as informações dos Clusters gerados.
Utilizei o Summary, extraindo dele somente a informação 'mean', para analisar cada um dos clusters separadamente.

In [41]:
# Desafio Extra! Avaliando as informações dos clusters gerados
print('Cluster 0')
dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 0).summary('mean').show()
print('Cluster 1')
dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 1).summary('mean').show()
print('Cluster 2')
dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 2).summary('mean').show()
print('Cluster 3')
dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 3).summary('mean').show()
print('Cluster 4')
dados_imoveis_kmeans.filter(dados_imoveis_kmeans.cluster_pca == 4).summary('mean').show()

Cluster 0
+-------+----+------------------+-----------------+------------------+-----------------+------------------+------------+------------+--------+------------------+------+-----------------+------------------+----+----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-----------+
|summary|  id|             andar|        area_util|         banheiros|          quartos|            suites|tipo_anuncio|tipo_unidade|tipo_uso|              vaga|bairro|       condominio|              iptu|tipo|           label|Condomínio fechado|        Playground| Portão eletrônico|           Piscina|Animais permitidos|      Portaria 24h|          Elevador|          Academia|   Salão de festas|     Churrasqueira|         Zona Norte|        Zona Oeste|       Zona Central|     

#Analisando as diferenças entre Clusters
Analisando do ponto de vista de área útil, podemos ver que os clusters podem ser separados com média mais baixa ou mais alta de área, o que sempre se reflete no preço ( 86 mil do Cluster 0 contra 359 mil do Cluster 3). O cluster 3 possui uma média de área útil de 280 m², maior, enquanto o cluster 0 possui a menor média, 90 m². 

Podemos pensar, baseados nisso, que os imóveis da Zona Oeste (maioria do Cluster 0), são em média menores e mais baratos do que os da Zona Sul (maioria do Cluster 3).

Realmente é possível passar horas analisando esses dados e tirando novas conclusões, mas isso, em minha opinião, reforça a qualidade do método do KMeans para agrupar imóveis por ele considerado parecidos e gerando estes clusters. Com certeza é uma ferramenta que irei utilizar muito em minha jornada como cientista de dados.

In [ ]:
spark.stop